In [1]:
import os
import sys
import torch
from torch.utils.tensorboard import SummaryWriter

In [2]:
sys.path.append('../')
from models.vgg import BACKBONE_LAYER_CONFIGS
from models.vgg import VGGEncoder

In [3]:
encoder = VGGEncoder(
    layer_config=BACKBONE_LAYER_CONFIGS['W5'],
    in_channels=2,
    output_size=(5, 5)
)

In [4]:
encoder.load_pretrained("../temp/best_model.denoising.00.pt", key='encoder')

<All keys matched successfully>

In [5]:
for child in encoder.children():
    print(child)

Sequential(
  (0): Conv2d(2, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): AdaptiveAvgPool2d(output_size=(5, 5))
)


In [6]:
from torchvision.transforms import Lambda

In [7]:
from datasets.wafer import Wafer40Dataset

In [8]:
test_set = Wafer40Dataset("../temp/wafer40.00.npz", mode='test')

In [9]:
features = encoder(torch.cat([test_set.x, test_set.m], dim=1))

In [10]:
features = features.view(features.size(0), -1)

In [11]:
writer = SummaryWriter(log_dir="../temp/runs/denoising/")

In [12]:
scaled_x = Lambda(lambda x: (x-0)/(2-0))(test_set.x)
writer.add_embedding(features, metadata=test_set.y, label_img=scaled_x)